### Generate an annotated dataset

Things to include:
- Continuous score for gene sets
    - Immune score
    - proliferation score
    - etc.
- Cell-cycle phase activity: scanpy.tl.score_genes_cell_cycle
- qc metrics: scanpy.pp.calculate_qc_metrics
    - total counts
    - n_genes
    - percent_mito
    - percent_ribo
    - percent_hb
